In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
from pyspark.sql import Row
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark.sql import functions as fn
from pyspark.ml import feature, regression, evaluation, Pipeline
import seaborn as sns
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.stat import Correlation
import matplotlib.pyplot as plt

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
# Reading the csv file in us_acci dataframe
us_acci = spark.read.csv(get_training_filename('US_Accidents.csv'), header = True, inferSchema = True)

In [5]:
# Shape of the spark dataframe

print('Shape is ',(us_acci.count(),len(us_acci.columns)))

Shape is  (981531, 49)


In [6]:
# Renaming the columns names which contain brackets for ease of usage

us_acci = us_acci.select('*').withColumnRenamed('Distance(mi)','Distance')\
    .withColumnRenamed('Temperature(F)', 'Temperature').withColumnRenamed('Wind_Chill(F)', 'Wind_Chill')\
    .withColumnRenamed('Humidity(%)', 'Humidity').withColumnRenamed('Pressure(in)', 'Pressure')\
    .withColumnRenamed('Visibility(mi)', 'Visibility').withColumnRenamed('Wind_Speed(mph)', 'Wind_Speed')\
    .withColumnRenamed('Precipitation(in)', 'Precipitation')

In [7]:
# Schema of dataframe with datatypes of all columns

us_acci.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- TMC: double (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Number: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Wind_Chill: double (nullable = 

In [8]:
# Summary of all numerical variables in the dataset to find the skewness, outliers, mean, median

us_acci.describe().toPandas()

In [8]:
# Columns to be dropped initially as a list
drop_col = ['ID','End_Lat','End_Lng','Description','Number','Street','Zipcode','Airport_Code','Country','Weather_Timestamp','Wind_Chill','Turning_Loop']

In [9]:
# Dropping columns from the original dataset

us_acci = us_acci.drop(*(drop_col))

In [10]:
# Checking null values in all the columns

from pyspark.sql.functions import isnan, isnull, when, count, col

us_acci.select([count(when(isnull(c), c)).alias(c) for c in us_acci.columns]).show()

+------+------+--------+----------+--------+---------+---------+--------+----+----+------+-----+--------+-----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+----+--------+--------+--------+-------+-------+----------+-------+----+---------------+--------------+--------------+--------------+-----------------+---------------------+
|Source|   TMC|Severity|Start_Time|End_Time|Start_Lat|Start_Lng|Distance|Side|City|County|State|Timezone|Temperature|Humidity|Pressure|Visibility|Wind_Direction|Wind_Speed|Precipitation|Weather_Condition|Amenity|Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station|Stop|Traffic_Calming|Traffic_Signal|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|
+------+------+--------+----------+--------+---------+---------+--------+----+----+------+-----+--------+-----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+----+--------+--

In [11]:
# Replacing same Wind_Direction named with different string N and North same

us_acci = us_acci.withColumn('Wind_Direction', when(us_acci.Wind_Direction == 'E', 'East')\
                   .when(us_acci.Wind_Direction == 'W', 'West')\
                   .when(us_acci.Wind_Direction == 'N', 'North')\
                   .when(us_acci.Wind_Direction == 'S', 'South')\
                   .when(us_acci.Wind_Direction == 'VAR', 'Variable')\
                   .when(us_acci.Wind_Direction == 'CALM', 'Calm')\
                   .otherwise(us_acci.Wind_Direction))

In [12]:
# Replacing same Weather_Condition named with different string Light Rain Shower and Light Rain Showers same

us_acci = us_acci.withColumn('Weather_Condition', when(us_acci.Weather_Condition == 'Light Rain Shower', 'Light Rain Showers')\
                   .when(us_acci.Weather_Condition == 'Light Snow Shower', 'Light Snow Showers')\
                   .when(us_acci.Weather_Condition == 'Rain Shower', 'Rain Showers')\
                   .otherwise(us_acci.Weather_Condition))

In [13]:
# Dropping Null rows from City Column as there are only 30 rows with City = Null

us_acci = us_acci.where(col("city").isNotNull())

In [14]:
# Dropping Null rows from Timezone Column 

us_acci = us_acci.where(col("Timezone").isNotNull())

In [15]:
# Clipping Temperature extreme values to suppress outliers

lower = -30
upper = 115
us_acci = us_acci.withColumn('Temperature', when(us_acci.Temperature > upper, upper)\
                  .when(us_acci.Temperature < lower, lower).otherwise(us_acci.Temperature).alias('Temperature'))

In [16]:
# Distance > 100 rows dropped
us_acci = us_acci.where(us_acci.Distance <100)

In [17]:
# Clipping Visibility extreme values to suppress outliers 

upper = 20
us_acci = us_acci.withColumn('Visibility', when(us_acci.Visibility > upper, upper)\
                  .otherwise(us_acci.Visibility).alias('Visibility'))

In [18]:
# Clipping Wind_Speed extreme values to suppress outliers 
upper = 40
us_acci = us_acci.withColumn('Wind_Speed', when(us_acci.Wind_Speed > upper, upper)\
                  .otherwise(us_acci.Wind_Speed).alias('Wind_Speed'))

In [19]:
# Replacing Null values in Precipitation with 0

us_acci = us_acci.withColumn('Precipitation', when(us_acci.Precipitation.isNull(), 0).otherwise(us_acci.Precipitation))

In [20]:
#replacing missing values in categorical attributes with the mode of the corresponding variables

for col_name in ['Wind_Direction', 'Weather_Condition']:
    common = us_acci.dropna().groupBy(col_name).agg(fn.count('*')).orderBy('count(1)', ascending = False).first()[col_name]
    us_acci = us_acci.withColumn(col_name, when(isnull(col_name), common).otherwise(us_acci[col_name]))

In [21]:
#replacing missing values in numerical attributes with the median of the corresponding variables

for col_name in ['Temperature', 'Humidity', 'Pressure', 'Visibility', 'Wind_Speed']:
    median = us_acci.dropna().approxQuantile(col_name, [0.5], 0.00)[0]
    us_acci = us_acci.withColumn(col_name, when(isnull(col_name), median).otherwise(us_acci[col_name]))

In [22]:

# Removing Null values in last 4 columns 

us_acci = us_acci.filter(us_acci.Sunrise_Sunset.isNotNull())

us_acci = us_acci.filter(us_acci.Civil_Twilight.isNotNull())

us_acci = us_acci.filter(us_acci.Nautical_Twilight.isNotNull())

us_acci = us_acci.filter(us_acci.Astronomical_Twilight.isNotNull())

In [23]:
# Adding Month of Year, Day of Week , Week of Year and Hour of the Day Column from Start Time

us_acci = us_acci.withColumn("Start_Time",to_timestamp(col("Start_Time"))).withColumn("month_of_year", date_format(col("Start_Time"), "MMMM")).withColumn("day_of_week", date_format(col("Start_Time"), "EEEE")).withColumn("hour_day", date_format(col("Start_Time"), "H")).withColumn("week_of_year", date_format(col("Start_Time"), "w"))

In [24]:
# Checking null values in all the columns

from pyspark.sql.functions import isnan, isnull, when, count, col

us_acci.select([count(when(isnull(c), c)).alias(c) for c in us_acci.columns]).show()

+------+------+--------+----------+--------+---------+---------+--------+----+----+------+-----+--------+-----------+--------+--------+----------+--------------+----------+-------------+-----------------+-------+----+--------+--------+--------+-------+-------+----------+-------+----+---------------+--------------+--------------+--------------+-----------------+---------------------+-------------+-----------+--------+------------+
|Source|   TMC|Severity|Start_Time|End_Time|Start_Lat|Start_Lng|Distance|Side|City|County|State|Timezone|Temperature|Humidity|Pressure|Visibility|Wind_Direction|Wind_Speed|Precipitation|Weather_Condition|Amenity|Bump|Crossing|Give_Way|Junction|No_Exit|Railway|Roundabout|Station|Stop|Traffic_Calming|Traffic_Signal|Sunrise_Sunset|Civil_Twilight|Nautical_Twilight|Astronomical_Twilight|month_of_year|day_of_week|hour_day|week_of_year|
+------+------+--------+----------+--------+---------+---------+--------+----+----+------+-----+--------+-----------+--------+------

In [25]:
# Shape of the spark dataframe

print('Shape is ',(us_acci.count(),len(us_acci.columns)))

Shape is  (980416, 41)


In [26]:
us_acci.toPandas().to_csv("Us_clean.csv",header=True,index=False)

# EDA

In [29]:
# Converting spark dataframe to Pandas DF for EDA

Us_acci_pd = us_acci.toPandas()

In [30]:
# Monthly Accident count showing how distribution of accidents on 12 months of the year

month_lst = ['January','February','March','April','May','June','July','August','September','October','November','December']
Us_acci_pd.groupby(['Severity', 'month_of_year']).size().reset_index().pivot(columns='Severity', index='month_of_year', values=0).reindex(month_lst).plot(kind='bar', stacked=True, title='Monthly Accident count for the year for each severity',)
display()

In [31]:
# Daily Accident count showing how distribution of accidents on 7 days of the week

weekday_lst = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Us_acci_pd.groupby(['Severity', 'day_of_week']).size().reset_index().pivot(columns='Severity', index='day_of_week',values=0).reindex(weekday_lst).plot(kind='bar', stacked=True,title='Daily Accident count for the week for each severity')
display()

In [32]:
# Day light savings in USA bi-annually for the weeks 11 and 45 rise is seen as compared to previous weeks 10 & 44 for March & November respectively

col = ['10','11','12','44','45','46']
newcol = Us_acci_pd[Us_acci_pd.week_of_year.isin(col)]

In [33]:
# For Day light Savings checking
day_acci = newcol.groupby(['Severity', 'week_of_year']).size().reset_index().pivot(columns='Severity', index='week_of_year', values=0).plot(kind='bar', stacked=True,title='Day Light Savings Week Comparison of Accidents')
display(day_acci)

In [34]:
subset=['Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight']

In [35]:
subset.remove("Turning_Loop")

In [36]:
# Severity as a percentage on y axis showing how severe the accidents can be considering the various variables 

fig,ax=plt.subplots(4,4,figsize=(20,30))
for i in range(len(subset)):
    df_plot = Us_acci_pd.groupby([subset[i], "Severity"]).size().reset_index().pivot(columns=subset[i], index="Severity", values=0)
    df_plot.div(df_plot.sum(axis=1), axis=0).plot(kind='bar', stacked=True,ax=ax[int(i/4),i-int(i/4)*4])
display()

In [37]:
# Sunrise Sunset for Day & Night comparison of Severity 

df_plot = Us_acci_pd.groupby(["Sunrise_Sunset","Civil_Twilight"]).size().reset_index().pivot(columns="Sunrise_Sunset", index= "Civil_Twilight", values=0)
display(df_plot.div(df_plot.sum(axis=1), axis=0).plot(kind='bar', stacked=True,title='Sunrise Sunset for Day & Night comparison of Severity'))

In [38]:
fig,ax=plt.subplots(1,figsize=(20,10))
sns.scatterplot(x='Start_Lng', y='Start_Lat', data=Us_acci_pd, hue='Severity',palette='RdYlGn_r',ax=ax)
ax.xlabel('Longitude')
ax.ylabel('Latitude)')
ax.set_title('US Map showing severity of accidents by their categories')
plt.show()
display()

In [39]:
# Accident count showing how distribution of accidents on each hour days of the day for the whole 24 hours

hour_lst = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']


display(Us_acci_pd.groupby(['Severity', 'hour_day']).size().reset_index().pivot(columns='Severity', index='hour_day', values=0).reindex(hour_lst).plot(kind='bar', stacked=True,figsize=(10,7),title='Hourly accident count for each severity throughout 24 hours of a day'))

In [40]:
# Accidents count by each state and for each state count by severity of accidents

display(pd.crosstab(columns=Us_acci_pd['Severity'],
    index=Us_acci_pd['State']).plot(kind='bar',stacked=True,figsize=(16,8),color=['purple','orange','blue','red','green'],title='State Wise Accident count for each severity along with total count'))

In [41]:
# Percent of severity
f,ax=plt.subplots(1,2,figsize=(12,8))
Us_acci_pd['Severity'].value_counts().plot.pie(explode=[0,0,0,0.2],autopct='%1.1f%%',ax=ax[0],shadow=False)
ax[0].set_title('Percentage Severity Distribution')
ax[0].set_ylabel('Count')
display(sns.countplot('Severity',data=Us_acci_pd,ax=ax[1],order=Us_acci_pd['Severity'].value_counts().index))
ax[1].set_title('Count of Severity')
plt.show()

In [42]:
# plot a bar plot to visualize to see which states have the highest number of accidents
top_ten_cities = Us_acci_pd['State'].value_counts().head(10)
display(top_ten_cities.plot.bar(width=0.5,edgecolor='k',align='center',linewidth=2,title='10 US States with the Highest Number of Accidents',figsize=(16,7)))
plt.xlabel('States',fontsize=20)
plt.ylabel('Number of Accidents',fontsize=20)
ax.tick_params(labelsize=20)
plt.grid()

In [43]:
# plot a bar plot to visualize to see which cities have the highest number of accidents
top_ten_cities = Us_acci_pd['City'].value_counts().head(10)
display(top_ten_cities.plot.bar(width=0.5,edgecolor='k',align='center',linewidth=2,title='10 US Cities with the Highest Number of Accidents',figsize=(16,7)))
plt.xlabel('Cities',fontsize=20)
plt.ylabel('Number of Accidents',fontsize=20)
ax.tick_params(labelsize=20)
plt.grid()

In [44]:
# Accidents based on the side of the driving Left & Right USA Right side driving 

Side_lst = ['R','L']
display(Us_acci_pd.groupby(['Severity', 'Side']).size().reset_index().pivot(columns='Severity', index='Side',values=0).reindex(Side_lst).plot(kind='bar', stacked=True,title='Accidents Severity based on Right or Left Side of Driving'))

In [45]:
display(Us_acci_pd.boxplot(column=['Distance']))

In [46]:
display(Us_acci_pd.boxplot(column=['Temperature']))

In [47]:
display(Us_acci_pd.boxplot(column=['Wind_Speed']))

In [48]:
display(Us_acci_pd.boxplot(column=['Humidity']))

In [49]:
display(Us_acci_pd.boxplot(column=['Pressure']))

In [50]:
display(Us_acci_pd.boxplot(column=['Visibility']))

In [51]:
# Univariate histogram of Temp 

var = 'Temperature'
x = Us_acci_pd[var]

bins = np.arange(-30, 120,10.0)

plt.hist(x, bins, alpha=0.8, histtype='bar', color='gold',
         ec='black')

plt.xlabel(var)
plt.ylabel('count')
plt.xticks(bins)
display(plt.show())


In [52]:
# Univariate Analysis of Wind_Speed

var = 'Wind_Speed'
x = Us_acci_pd[var]

bins = np.arange(0, 100,5.0)

plt.hist(x, bins, alpha=0.8, histtype='bar', color='gold',
         ec='black')

plt.xlabel(var)
plt.ylabel('count')
plt.xticks(bins)
display(plt.show())